### 2D Confocal Mitochondria–Spine Measurements

**Goal:**  
Measure mitochondrial length, presence under spines, mitochondrial area within 1 µm of spines, spine area, and high-width mitochondrial regions.

**Experimental context:**  
3D confocal stacks of mitochondria and PSD95 were upscaled and Huygens-deconvolved, then maximum-intensity projected. Dendrites were traced on the merged projections in Fiji, and both projections and dendrite traces (CSV) served as inputs.

**Processing steps:**  
1. **Dendrite reconstruction** – Builds hierarchical dendritic trees from traced CSV files.  
2. **PSD95 puncta detection** – Segments and measures spine areas.  
3. **Mitochondrial segmentation** – Generates masks, skeletons, and local width maps, identifying high-width remodeled regions.  
4. **Mito–spine coupling** – Quantifies mitochondrial length, presence under spines, and mitochondrial area within 1 µm of each spine.  
5. **Density profiling** – Calculates bin-wise density profiles for mitochondria and PSD95 puncta along dendrites.

> **Note:** All image processing and measurements operate in **pixel units**. Function parameters (e.g., radius, bin size, step size) must therefore be specified in pixels.

**Outputs:**  
- **Images:** Mitochondrial and PSD95 masks, mitochondrial skeleton, width and high-width maps, and overlays of mitochondria under spines.  
- **Tables (CSV):** Mitochondrial length, spine area, mitochondrial area within 1 µm of spines, and high-width mitochondrial region metrics.


In [ ]:
from Confocal2D import process, visualize
import tifffile
from tifffile import imwrite
from skimage import io
import os
import numpy as np


# Base directory paths
base_dir = r"./Demo Images"
dendrites_dir = os.path.join(base_dir, "Dendrites")
psd95_dir = os.path.join(base_dir, "PSD95Max")
mito_dir = os.path.join(base_dir, "MitoMax")
output_dir = os.path.join(base_dir, "Morphometrics")

# Ensure the output directory exists
os.makedirs(output_dir, exist_ok=True)

# Process each cell
for csv_file in os.listdir(dendrites_dir):
    if csv_file.endswith(".csv"):
        # Extract the base name without the extension
        base_name = csv_file.replace("_merge.tif.csv", "")  # Remove the unique part of the dendrite filename

        # Construct paths for the current cell
        csv_path = os.path.join(dendrites_dir, csv_file)
        mito_image_path = os.path.join(mito_dir, f"{base_name}.tif")
        psd95_image_path = os.path.join(psd95_dir, f"{base_name.replace('_C0_', '_C1_')}.tif")

        # Verify corresponding PSD95 and Mito files exist
        if not os.path.exists(psd95_image_path) or not os.path.exists(mito_image_path):
            print(f"Missing files for {base_name}. Skipping...")
            continue

        # Create an output directory for this cell
        cell_output_dir = os.path.join(output_dir, base_name)
        os.makedirs(cell_output_dir, exist_ok=True)

        # Load images
        PSD95_image = io.imread(psd95_image_path)
        Mito_image = io.imread(mito_image_path)

        save_image = True

        #Generate dendritic tree
        root_nodes, dendrites = process.make_tree(csv_path, prune=True)
        
        visualize.tree_map(root_nodes, dendrites, PSD95_image, save_image=save_image, 
                           output_filename=os.path.join(cell_output_dir, f"{base_name}_Dendritic_Tree.tif"), 
                           dpi=300)
        
        PSD95_puncta, PSD95_mask = process.detect_puncta(
            PSD95_image, 
            root_nodes, 
            dendrites, 
            radius_dendrite=50, 
            min_sigma=1.0, 
            max_sigma=1.5, 
            num_sigma=10, 
            threshold_factor=0.8, 
            min_size=0)
        
        visualize.mask_overlay(PSD95_image, PSD95_mask, dpi=300, save_image=True, 
                               output_filename=os.path.join(cell_output_dir, f"{base_name}_PSD95_mask_overlay.tif"), 
                               mode="solid")
        
        tifffile.imwrite(os.path.join(cell_output_dir, f"{base_name}_PSD95_mask.tif"), (PSD95_mask.astype(np.uint8) * 255), dtype=np.uint8)
        
        Mito_mask = process.detect_objects(Mito_image,
            root_nodes,
            dendrites,
            radius=25,
            bin_size=100,
            overlap_radius=25,
            min_overlap=0.8,
            min_size=50
        )
        
        visualize.mask_overlay(Mito_image, Mito_mask, dpi=300, save_image=True, 
                               output_filename=os.path.join(cell_output_dir, f"{base_name}_Mito_mask_overlay.tif"), 
                               mode="solid")
        
        imwrite(os.path.join(cell_output_dir, f"{base_name}_Mito_mask.tif"), (Mito_mask.astype(np.uint8) * 255))
            
        width_image, remodeled_width_image,  mito_skel, remodeled_data = process.remodeled_mito_width(
            Mito_mask, root_nodes, dendrites, std_multiplier=1.0,
            skeleton_threshold=10, min_region_size=0, step=1
        )
        
        remodeled_data.to_csv(os.path.join(cell_output_dir, f"{base_name}_remodeled_mito.csv"), index=False)
        
        imwrite(os.path.join(cell_output_dir, f"{base_name}_mito_skeleton.tif"), (mito_skel.astype(np.uint8) * 255))
        
        
        Remodeled_mito_mask = (remodeled_width_image > 0).astype(np.uint8)
        
        visualize.thickness_image(Mito_mask, width_image,
                                  save_image=True,
                                  output_filename=os.path.join(cell_output_dir, f"{base_name}_mito_width.tif"))
             
        visualize.thickness_image(Mito_mask, remodeled_width_image,
                                  save_image=True,
                                  output_filename=os.path.join(cell_output_dir, f"{base_name}_remodeled_mito.tif"))
             
        mito_under = process.mito_under_synapse(PSD95_puncta, PSD95_image, Mito_mask, dendrites,rect_length=30, rect_width=20)
        mito_under.to_csv(os.path.join(cell_output_dir, f"{base_name}_mito_under.csv"), index=False)
        visualize.mito_under_synapse_nonoverlapping(
            dataframe=mito_under,
            Mito_mask=Mito_mask,
            min_distance=0,
            save_image=True,
            output_filename=os.path.join(cell_output_dir, f"{base_name}_mito_under.tif"))
        
        
        remodeled_mito_under = process.mito_under_synapse(PSD95_puncta, PSD95_image, Remodeled_mito_mask, dendrites,rect_length=30, rect_width=20)
        remodeled_mito_under.to_csv(os.path.join(cell_output_dir, f"{base_name}_remodeled_mito_under.csv"), index=False)
        
        visualize.mito_under_synapse_nonoverlapping(
            dataframe=remodeled_mito_under,
            Mito_mask=Remodeled_mito_mask,
            min_distance=0,
            save_image=True,
            output_filename=os.path.join(cell_output_dir, f"{base_name}_remodeled_mito_under.tif"))
        
        
        density_profiles, interpolated_dendrites = process.calculate_density_profiles(
            PSD95_mask, Mito_mask, dendrites, root_nodes,
            resolution=1.0, bin_size=100, step_size=10
        )
        density_profiles.to_csv(os.path.join(cell_output_dir, f"{base_name}_density_profiles.csv"), index=False)
        
        density_profiles, interpolated_dendrites = process.calculate_density_profiles(
            PSD95_mask, Remodeled_mito_mask, dendrites, root_nodes,
            resolution=1.0, bin_size=100, step_size=10
        )
        density_profiles.to_csv(os.path.join(cell_output_dir, f"{base_name}_remodeled_density_profiles.csv"), index=False)
        print(f"Processing complete for {base_name}.")


print("Batch processing completed successfully.")